# 映画の推薦 予測&生成AI デモ

In [1]:
import requests
import json

import pandas as pd
import numpy as np

from h2ogpte import H2OGPTE

In [32]:
MLOPS_ENDPOINT = 'https://model.internal.dedicated.h2o.ai/98f52303-8408-41d1-ac9b-b6c45506aa3e/model/score'

H2OGPTE_URL = 'https://playground.h2ogpte.h2o.ai'

with open('../_secret') as f:
    key = json.load(f)
H2OGPTE_KEY = key['movie_sample2']

## 顧客に対する推薦リストの作成 - Predictive AI

In [5]:
!ls data

movies_sample.csv                   user_demographic_and_preference.csv


In [15]:
df_customer = pd.read_csv('data/user_demographic_and_preference.csv')
print(df_customer.shape)
df_customer = df_customer.sample(n=1)

(15, 18)


**推薦を実施したい顧客**
- user_* : 顧客情報
- genre_* : 好きな映画のジャンル

In [14]:
df_customer

,userId,user_gender,user_generation,genre_Action,genre_Adventure,genre_Animation,genre_Children,genre_Comedy,genre_Crime,genre_Drama,genre_Fantasy,genre_Horror,genre_IMAX,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Thriller,genre_War
5,10006,M,Senior,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


**推薦を実施する映画のリスト**

In [20]:
df_movie = pd.read_csv('data/movies_sample.csv')
df_movie = df_movie.drop(columns='genres')
df_movie

,movieId,title
0,1,Toy Story (1995)
1,47,Seven (a.k.a. Se7en) (1995)
2,50,"Usual Suspects, The (1995)"
3,110,Braveheart (1995)
4,150,Apollo 13 (1995)
5,260,Star Wars: Episode IV - A New Hope (1977)
6,296,Pulp Fiction (1994)
7,318,"Shawshank Redemption, The (1994)"
8,356,Forrest Gump (1994)
9,480,Jurassic Park (1993)


**スコアリング用データ（MLOpsへのリクエスト）**  
- 推薦を実施するUserIdに対し、映画の情報（movieId）を紐づける
- userIdとtitleはスコアリングに必要ないが、見やすさのため

In [23]:
df_scoring = pd.merge(df_customer, df_movie, how='cross')
print(df_scoring.shape)
df_scoring.head()

(20, 20)


,userId,user_gender,user_generation,genre_Action,genre_Adventure,genre_Animation,genre_Children,genre_Comedy,genre_Crime,genre_Drama,genre_Fantasy,genre_Horror,genre_IMAX,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Thriller,genre_War,movieId,title
0,10013,M,Senior,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,Toy Story (1995)
1,10013,M,Senior,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,47,Seven (a.k.a. Se7en) (1995)
2,10013,M,Senior,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,50,"Usual Suspects, The (1995)"
3,10013,M,Senior,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,110,Braveheart (1995)
4,10013,M,Senior,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,150,Apollo 13 (1995)


In [25]:
def transform_scoringdata(df) -> dict:
    
    # jsonに変換
    json_data = df.to_json(orient="split")
    json_data = json.loads(json_data)
    # キー名の変更
    json_data['fields'] = json_data['columns']
    del json_data['columns']
    json_data['rows'] = json_data['data']
    del json_data['data']
    
    return json_data

In [28]:
scoring_data = transform_scoringdata(df_scoring)
type(scoring_data)

dict

In [30]:
scoring_data.keys()

dict_keys(['index', 'fields', 'rows'])

In [33]:
response = requests.post(url=MLOPS_ENDPOINT, 
                         headers={'content-type': 'application/json'}, 
                         data=json.dumps(scoring_data))
response

<Response [200]>

In [54]:
def transform_scoringdata(response) -> pd.DataFrame:
    response_df = pd.concat([df_scoring, pd.DataFrame(response.json()['score'])], axis=1)
    response_df = response_df.rename(columns={1:'score'})
    response_df = response_df[['userId', 'user_gender', 'user_generation', 'movieId', 'title', 'score']]
    response_df = response_df.sort_values('score', ascending=False)
    return response_df

In [57]:
df_recomendation_list = transform_scoringdata(response)
df_recomendation_list

,userId,user_gender,user_generation,movieId,title,score
18,10013,M,Senior,2959,Fight Club (1999),0.4479908206399705
19,10013,M,Senior,4993,"Lord of the Rings: The Fellowship of the Ring,...",0.4367456918400951
16,10013,M,Senior,2571,"Matrix, The (1999)",0.4334245643699235
14,10013,M,Senior,1196,Star Wars: Episode V - The Empire Strikes Back...,0.42149340938190794
5,10013,M,Senior,260,Star Wars: Episode IV - A New Hope (1977),0.4033679403459303
2,10013,M,Senior,50,"Usual Suspects, The (1995)",0.40298541566227036
15,10013,M,Senior,1198,Raiders of the Lost Ark (Indiana Jones and the...,0.3843803993797101
7,10013,M,Senior,318,"Shawshank Redemption, The (1994)",0.3832757883085894
17,10013,M,Senior,2858,American Beauty (1999),0.38279121506913516
6,10013,M,Senior,296,Pulp Fiction (1994),0.3716957923137419


## 顧客に対する推薦文の作成 - Generative AI

参考: https://github.com/yukismd/H2O_LLM/blob/main/h2oGPTe/Collection-specific-API_RAG.ipynb

In [96]:
!ls data

movies_sample.csv                   user_demographic_and_preference.csv
movies_sample_story.txt


RAG用ドキュメント: ./data/movies_sample_story.txt （各映画の詳細情報）

In [58]:
client = H2OGPTE(
    address=H2OGPTE_URL,
    api_key=H2OGPTE_KEY,
)

client

In [83]:
client.get_llm_names()

['mistralai/Mixtral-8x7B-Instruct-v0.1',
 'meta-llama/Meta-Llama-3-8B-Instruct',
 'meta-llama/Meta-Llama-3-70B-Instruct',
 'nvidia/Llama3-ChatQA-1.5-70B',
 'NousResearch/Nous-Capybara-34B',
 'mistralai/Mistral-7B-Instruct-v0.3',
 'h2oai/h2o-danube2-1.8b-chat',
 'mistralai/Mistral-Nemo-Instruct-2407',
 'OpenGVLab/InternVL-Chat-V1-5',
 'THUDM/cogvlm2-llama3-chat-19B',
 'liuhaotian/llava-v1.6-34b',
 'lmms-lab/llama3-llava-next-8b',
 'mistral-small-latest',
 'mistral-large-latest',
 'mistral-medium',
 'claude-3-5-sonnet-20240620',
 'claude-3-sonnet-20240229',
 'claude-3-opus-20240229',
 'claude-3-haiku-20240307',
 'microsoft/Phi-3-vision-128k-instruct',
 'microsoft/Phi-3-medium-128k-instruct',
 'google/gemma-2-27b-it',
 'gemini-1.5-pro-latest',
 'gemini-1.5-flash-latest',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-16k-0613',
 'gpt-35-turbo-1106',
 'gpt-4-1106-preview',
 'gpt-4-turbo-2024-04-09']

In [60]:
chat_session_id = client.create_chat_session_on_default_collection()
chat_session_id

'18e4f762-551d-4060-a8be-f762fcd0b092'

**顧客属性**

In [62]:
user_gender = df_recomendation_list['user_gender'][0]
user_generation = df_recomendation_list['user_generation'][0]
user_gender, user_generation

('M', 'Senior')

In [82]:
user_gender_prompt = {'M':'男性', 'F':'女性'}.get(user_gender)
user_generation_prompt = {'Young':'10~20代', 'Middle':'30~40代', 'Senior':'50~60代'}.get(user_generation)
user_gender_prompt, user_generation_prompt

('男性', '50~60代')

**推薦映画 - 上位2つ**

In [65]:
movie_top1 = df_recomendation_list['title'][0]
movie_top2 = df_recomendation_list['title'][1]
movie_top1, movie_top2

('Toy Story (1995)', 'Seven (a.k.a. Se7en) (1995)')

In [93]:

Message = '''
Movie Title = {2}とMovie Title = {3}を{1}の{0}におすすめして下さい。
文章には以下の３点を含めてください。

- {1}の{0}に対するフレンドリーなオープニングトーク。
- 映画の要約。100文字以内。
    - Movie Title = {2}のMovie Detailを参考にした100文字以内の要約文。
    - Movie Title = {3}のMovie Detailを参考にした100文字以内の要約文。
- なぜ{1}の{0}にそれらの映画がおすすめかを説明
'''.format(user_gender_prompt, user_generation_prompt, movie_top1, movie_top2)

print(Message)


Movie Title = Toy Story (1995)とMovie Title = Seven (a.k.a. Se7en) (1995)を50~60代の男性におすすめして下さい。
文章には以下の３点を含めてください。

- 50~60代の男性に対するフレンドリーなオープニングトーク。
- 映画の要約。100文字以内。
    - Movie Title = Toy Story (1995)のMovie Detailを参考にした100文字以内の要約文。
    - Movie Title = Seven (a.k.a. Se7en) (1995)のMovie Detailを参考にした100文字以内の要約文。
- なぜ50~60代の男性にそれらの映画がおすすめかを説明



In [95]:
LlmModel = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

SysPrompt = '''
あなたは映画のおすすめ情報を作成するAIです。（回答は必ず日本語で実施します。）
'''

Message = '''
Movie Title = {2}とMovie Title = {3}を{1}の{0}におすすめして下さい。
文章には以下の３点を含めてください。

- {1}の{0}に対するフレンドリーなオープニングトーク。
- 映画の要約。100文字以内。
    - Movie Title = {2}のMovie Detailを参考にした100文字以内の要約文。
    - Movie Title = {3}のMovie Detailを参考にした100文字以内の要約文。
- なぜ{1}の{0}にそれらの映画がおすすめかを説明
'''.format(user_gender_prompt, user_generation_prompt, movie_top1, movie_top2)


print('>>>>> Prompt')
print(Message)

print('>>>>> LLM Reply')
with client.connect(chat_session_id) as session:
    reply = session.query(
        message = Message,
        system_prompt=SysPrompt,
        llm=LlmModel,
        rag_config={"rag_type": "rag"},   # (Normal) RAG 
    )
    print(reply.content)

>>>>> Prompt

Movie Title = Toy Story (1995)とMovie Title = Seven (a.k.a. Se7en) (1995)を50~60代の男性におすすめして下さい。
文章には以下の３点を含めてください。

- 50~60代の男性に対するフレンドリーなオープニングトーク。
- 映画の要約。100文字以内。
    - Movie Title = Toy Story (1995)のMovie Detailを参考にした100文字以内の要約文。
    - Movie Title = Seven (a.k.a. Se7en) (1995)のMovie Detailを参考にした100文字以内の要約文。
- なぜ50~60代の男性にそれらの映画がおすすめかを説明

>>>>> LLM Reply
こんにちは！今日は素晴らしい映画をお勧めしたいと思います。

最初に、『トイ・ストーリー』（1995）という名作アニメをご紹介します。主人公は、おもちゃのカウボーイであるウッディで、新しくやってきたおもちゃのバズと出会い、彼らとの冒険を経験します。この映画は、子供たちから大人まで幅広い層に愛され、絵画の質と緻密なストーリーテリングが魅力的です。

次に、『セブン』（1995）というミステリーとスリラーのジャンルを持つ名作をご紹介します。犯罪捜査官のサマセットとミルズが、7 つの大罪に基づいた猟奇的な殺人事件を解決しようとします。映画は、暗く陰鬱な雰囲気が漂う映像美と、ブラッド・ピットやモーガン・フリーマンの優れた演技が特徴的です。

これらの映画をお勧めする理由は、50~60代の男性にとって、熟練した演技や深いストーリーテリングが好まれると考えます。特に、『トイ・ストーリー』は、子供たちにも大人たちにも楽しめる、心から笑えるコメディ要素と、絵画の質の高いアニメーションが魅力的です。一方、『セブン』は、犯罪と人間の心理を描いた深いストーリーが好まれる方にとっては、見逃せない一品です。

今回のお勧め映画を見て、心に残る感動を与えてくれることでしょう。ぜひ、家族や友人と一緒に楽しんでください。
